In [9]:
import os
from esa_snappy import ProductIO, HashMap, GPF, jpy
import esa_snappy
import numpy as np
import logging

# 目标目录
directory = r"F:\GoogleDrive\sentinel\LZT zips"

# 配置日志记录
# 配置日志记录
logging.basicConfig(
    filename=os.path.join(directory, 'tifs', 'error_log.txt'),
    level=logging.ERROR,       # 设置日志级别为 ERROR
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# 获取目录下所有ZIP文件的文件名
zip_files = [f for f in os.listdir(directory) if f.endswith('.zip')]

zip_files[0].split(".")[0]


'S3B_SL_2_LST____20230727T145505_20230727T145805_20230727T165501_0179_082_139_2340_PS2_O_NR_004'

In [10]:
err_list=[]
for zipfile in zip_files:
    try:
        # 读取
        # WKTReader = jpy.get_type('org.locationtech.jts.io.WKTReader')
        # wkt="POLYGON ((-74.255591 40.496134,-74.255591 40.915533,-73.700009 40.915533,-73.700009 40.496134,-74.255591 40.496134))"
        # geom = WKTReader().read(wkt)
        # params = HashMap()
        # params.put('geometryRegion',geom)
        # params.put('useAdvancedOptions',True)
        p = ProductIO.readProduct(f'{directory}\{zipfile}')
        # 裁剪
        WKTReader = jpy.get_type('org.locationtech.jts.io.WKTReader')
        wkt="POLYGON ((-74.255591 40.496134,-74.255591 40.915533,-73.700009 40.915533,-73.700009 40.496134,-74.255591 40.496134))"
        geom = WKTReader().read(wkt)
        params_subset = HashMap()
        params_subset.put('copyMetadata', True)
        params_subset.put('geoRegion', geom)
        subset = GPF.createProduct('Subset', params_subset, p)

        #重投影
        # band_names = ['LST', 'LST_uncertainty', 'bayes_in']
        param_rprj=HashMap()
        param_rprj.put("crs", "EPSG:32618")
        param_rprj.put("resampling", "Nearest")
        param_rprj.put("orthorectify", False)
        param_rprj.put("noDataValue", float('nan'))
        param_rprj.put("includeTiePointGrids", True)
        # param_rprj.put('sourceBands', ','.join(band_names))
        reproject = GPF.createProduct('Reproject',param_rprj,subset)

        # 去云mask: recommended to apply the probabilistic cloud mask (‘single_moderate’ bit in the Bayes variable of the Flags datafile)
        params = HashMap()
        BandDescriptor = jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')
        targetBand = BandDescriptor()
        targetBand.name = 'Mask'
        targetBand.type = 'float32'
        targetBand.expression = 'if bayes_in_single_moderate == 0 then LST else NaN'
        targetBands = jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', 1)
        targetBands[0] = targetBand
        params.put('targetBands', targetBands)
        product = GPF.createProduct('BandMaths', params, reproject)

        outpath_name = os.path.join(directory, 'tifs', f'{zipfile.split(".")[0]}.tif')
        #不支持更新数据
        incremental = False
        esa_snappy.GPF.writeProduct(product, esa_snappy.File(outpath_name), 'GeoTIFF', incremental, esa_snappy.ProgressMonitor.NULL)
        print('Product successfully saved in:', outpath_name)
    except:
        err_list.append(zipfile)
err_list


Product successfully saved in: F:\GoogleDrive\sentinel\LZT zips\tifs\S3B_SL_2_LST____20230727T145505_20230727T145805_20230727T165501_0179_082_139_2340_PS2_O_NR_004.tif
Product successfully saved in: F:\GoogleDrive\sentinel\LZT zips\tifs\S3A_SL_2_LST____20230729T144138_20230729T144438_20230730T233141_0180_101_310_2340_PS1_O_NT_004.tif
Product successfully saved in: F:\GoogleDrive\sentinel\LZT zips\tifs\S3B_SL_2_LST____20230908T144007_20230908T144307_20230908T164123_0179_083_367_2340_PS2_O_NR_004.tif
Product successfully saved in: F:\GoogleDrive\sentinel\LZT zips\tifs\S3B_SL_2_LST____20230907T150618_20230907T150918_20230907T170939_0179_083_353_2340_PS2_O_NR_004.tif
Product successfully saved in: F:\GoogleDrive\sentinel\LZT zips\tifs\S3A_SL_2_LST____20230905T145636_20230905T145936_20230907T000204_0179_103_082_2340_PS1_O_NT_004.tif
Product successfully saved in: F:\GoogleDrive\sentinel\LZT zips\tifs\S3A_SL_2_LST____20230729T144138_20230729T144438_20230729T164611_0179_101_310_2340_PS1_O_NR_

['S3A_SL_2_LST____20230727T153100_20230727T153400_20230729T001726_0179_101_282_2160_PS1_O_NT_004.SEN3.zip',
 'S3A_SL_2_LST____20220807T151139_20220807T151439_20220809T003211_0179_088_239_2340_PS1_O_NT_004.SEN3.zip',
 'S3A_SL_2_LST____20220719T150411_20220719T150711_20220721T002325_0179_087_353_2340_PS1_O_NT_004.SEN3.zip',
 'S3A_SL_2_LST____20220714T153108_20220714T153408_20220716T003644_0179_087_282_2160_PS1_O_NT_004.SEN3.zip',
 'S3A_SL_2_LST____20190731T145403_20190731T163503_20190801T223735_6059_047_296______LN2_O_NT_003.SEN3.zip',
 'S3B_SL_2_LST____20190730T144041_20190730T162141_20190731T204213_6059_028_139______LN2_O_NT_003.SEN3.zip',
 'S3A_SL_2_LST____20190730T152014_20190730T170114_20190731T223451_6059_047_282______LN2_O_NT_003.SEN3.zip',
 'S3B_SL_2_LST____20190729T150652_20190729T164752_20190730T211457_6059_028_125______LN2_O_NT_003.SEN3.zip',
 'S3A_SL_2_LST____20190728T143137_20190728T161236_20190729T203025_6059_047_253______LN2_O_NT_003.SEN3.zip',
 'S3B_SL_2_LST____20200722T1

In [12]:
import csv

# 设置输出的 CSV 文件路径
output_file = os.path.join(directory, 'tifs', 'error.csv')

# 写入 CSV 文件
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for error in err_list:
        writer.writerow([error])  # 将每个错误作为一行写入


In [14]:
# for /r "G:\我的云端硬盘\LST data\sentinel\LZT zips" %X in (*.zip) do (gpt NdviOp -Ssource=%X -PnirSourceBand="B8"  -PredSourceBand="B4" -t "E:\Downloads\tifs\%~nX.tif")

'S3A_SL_2_LST____20170517T142357_20170517T160456_20180929T235123_6059_017_367______LR1_R_NT_003.SEN3.zip'